In [1]:
pip install numpy opencv-python pyautogui


You should consider upgrading via the '/Users/tim/myenv/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install pynput

  Using cached pynput-1.7.6-py2.py3-none-any.whl (89 kB)
You should consider upgrading via the '/Users/tim/myenv/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [9]:
import cv2
import numpy as np
import pyautogui
import time
import random

def right_click_random_position():
    screen_width, screen_height = pyautogui.size()
    x = random.randint(0, screen_width - 1)
    y = random.randint(0, screen_height - 1)
    pyautogui.moveTo(x, y, duration=0.5)
    pyautogui.click(button='right')

def find_and_click(icons_paths, threshold=0.8):
    sup_clicked = False
    yumi_clicked = False
    can_click_lock = False  
    keep_running = False  
    last_clicked_times = {}  
    cooldown = 5  

    while True:
        screenshot = pyautogui.screenshot()
        screen = np.array(screenshot)
        screen_gray = cv2.cvtColor(screen, cv2.COLOR_BGR2GRAY)

        for icon_path in icons_paths:
            icon = cv2.imread(icon_path, 0)
            if icon is None:
                print(f"Error loading image: {icon_path}")
                continue

            res = cv2.matchTemplate(screen_gray, icon, cv2.TM_CCOEFF_NORMED)
            loc = np.where(res >= threshold)

            for pt in zip(*loc[::-1]):
                icon_name = icon_path.split('/')[-1]
                logical_x = (pt[0] + icon.shape[1]//2) // 2
                logical_y = (pt[1] + icon.shape[0]//2) // 2
                current_time = time.time()

                if icon_name in last_clicked_times and current_time - last_clicked_times[icon_name] < cooldown:
                    print(f"{icon_name} is still in cooldown.")
                    continue

                pyautogui.moveTo(logical_x, logical_y, duration=0.5)
                pyautogui.click(button='left')
                last_clicked_times[icon_name] = current_time

                if icon_path.endswith('sup.png'):
                    sup_clicked = True
                elif icon_path.endswith('yumi.png'):
                    yumi_clicked = True

                if sup_clicked and yumi_clicked:
                    can_click_lock = True  

                if 'lock.png' in icon_path and can_click_lock:
                    pyautogui.moveTo(logical_x, logical_y)
                    pyautogui.click(button='left')
                    print(f"Clicked lock.png at ({logical_x}, {logical_y}) after both sup and yumi clicked.")
                    sup_clicked = False  
                    yumi_clicked = False  
                    can_click_lock = False 
                    keep_running = True  
                elif icon_path.endswith('finish game.png') or icon_path.endswith('next.png'):
                    keep_running = False  

        if keep_running:
            right_click_random_position()

        #time.sleep(1)  

icons_paths = [
    '/Users/tim/Desktop/one more game.png',
    '/Users/tim/Desktop/continue.png',
    '/Users/tim/Desktop/find match1.png',
    '/Users/tim/Desktop/sup.png',
    '/Users/tim/Desktop/yumi1.png',
    '/Users/tim/Desktop/lock.png',
    '/Users/tim/Desktop/finish game.png',
    '/Users/tim/Desktop/next.png',
    '/Users/tim/Desktop/accept match.png',
    '/Users/tim/Desktop/ok.png',
]
find_and_click(icons_paths)


Clicked at (1089, 1508)


In [ ]:
from pynput import keyboard

# 定義鍵盤按下事件處理函數
def on_press(key):
    try:
        # 監聽到按鍵按下事件，輸出按下的按鍵
        print('按下按鍵: {0}'.format(key.char))
    except AttributeError:
        # 如果是特殊按鍵，輸出特殊按鍵的名稱
        print('特殊按鍵按下: {0}'.format(key))

# 定義鍵盤釋放事件處理函數
def on_release(key):
    # 監聽到按鍵釋放事件，輸出釋放的按鍵
    print('釋放按鍵: {0}'.format(key))
    # 如果按下Esc按鍵，則停止監聽
    if key == keyboard.Key.esc:
        return False

# 創建一個監聽器來監聽鍵盤事件
with keyboard.Listener(on_press=on_press, on_release=on_release) as listener:
    # 監聽器在一個線程中運行，這裡使用join函數阻塞主線程，直到監聽器停止
    listener.join()

